# Menganalisis Risiko Gagal Bayar Peminjam

Tugas Anda adalah menyiapkan laporan untuk divisi kredit suatu bank. Anda akan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Pihak bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporan Anda akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.

**Tujuan Proyek:**

Menghitung probabilitas gagal bayar dalam pelunasan pinjaman nasabah divisi kredit suatu bank berdasarkan beberapa karakteristik yang dimiliki oleh nasabah, seperti status perkawinan, tingkat pendapatan, tujuan pinjaman, dan jumlah anak yang dimilikinya.

**Hipotesis:**
1. Jumlah anak yang dimiliki oleh nasabah dapat mempengaruhi probabilitas gagal bayar dalam pelunasan pinjaman. Semakin banyak anak yang dimiliki, probabilitasnya semakin tinggi.

2. Status perkawinan biasanya berpengaruh terhadap stabilitas keuangan nasabah sehingga nasabah dengan status sudah menikah dapat memiliki probabilitas gagal bayar lebih rendah dibanding yang belum menikah atau hidup sendiri.

3. Nasabah dengan tingkat pendapatan rendah dapat mempunyai probabilitas gagal bayar yang lebih besar dibandingkan dengan nasabah dengan tingkat pendapatan tinggi.

4. Tujuan pinjaman untuk kebutuhan/investasi jangka panjang (contoh: membeli rumah atau *property*) dapat mempunyai probabilitas gagal bayar yang lebih besar dibandingkan dengan nasabah yang memiliki tujuan pinjaman untuk kebutuhan jangka pendek (contoh: untuk biaya menikah).

In [1]:
import pandas as pd

df=pd.read_csv('/datasets/credit_scoring_eng.csv')
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Eksplorasi data

**Deskripsi Data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - pengidentifikasi untuk status perkawinan nasabah
- `family_status_id` - tanda pengenal status perkawinan
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah pernah melakukan gagal bayar pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman


In [2]:
df.shape

(21525, 12)

Dataset ini memiliki 21525 baris dan 12 kolom

In [3]:
df.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Terdapat nilai negatif pada data di kolom `days_employed`. Seharusnya nilai pada kolom tersebut tidak mungkin negatif.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Berdasarkan informasi data diatas, terdapat 2 kolom yang memiliki nilai yang hilang, yaitu kolom `days_employed` dan `total_income`, karena total baris pada kedua kolom tersebut tidak sesuai dengan total baris keseluruhan pada dataset.

In [6]:
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [7]:
df['days_employed'].isna().sum()

2174

In [8]:
df['total_income'].isna().sum()

2174

Setelah memfilter kolom `days_employed` dan dilakukan penghitungan, berdasarkan informasi pada dataset, maka dapat disimpulkan jumlah baris dalam tabel yang telah difilter sesuai dengan jumlah nilai yang hilang, dengan penjelasan sebagai berikut:
- jumlah data non-null adalah 19351
- jumlah data dengan nilai yang hilang adalah 2174
- jika kedua nilai tersebut dijumlahkan, maka akan didapatkan total 21525. Jumlah ini sudah sesuai dengan jumlah baris pada dataset.

Nilai yang hilang pada kolom `days_employed` juga tampak simetris dengan kolom `total_income`.

In [10]:
df[(df['days_employed'].isna()) & (df['total_income'].isna())].shape

(2174, 12)

In [11]:
df['days_employed'].value_counts(dropna=False)

 NaN            2174
-1580.622577       1
-4122.460569       1
-2828.237691       1
-2636.090517       1
                ... 
-7120.517564       1
-2146.884040       1
-881.454684        1
-794.666350        1
-3382.113891       1
Name: days_employed, Length: 19352, dtype: int64

In [12]:
df['total_income'].value_counts(dropna=False)

NaN          2174
17312.717       2
31791.384       2
42413.096       2
22435.069       1
             ... 
23834.534       1
26124.613       1
28692.182       1
28477.783       1
41428.916       1
Name: total_income, Length: 19349, dtype: int64

In [13]:
missing_values=df['days_employed'].isna().sum()
total=len(df)
percentage=missing_values/total

print(f'{percentage:.0%}')

10%


**Kesimpulan sementara**

Setelah melakukan filter dan melihat jumlah baris pada tabel yang telah difilter, maka didapatkan kesimpulan bahwa jumlah baris dalam tabel yang telah difilter sesuai dengan jumlah nilai yang hilang. 

Total persentase nilai yang hilang adalah 10% dari dataset. Jumlah ini merupakan angka yang cukup besar dan dapat berpengaruh terhadap penilitian data. Namun perlu dilakukan penyelidikan lebih lanjut mengenai data yang hilang tersebut, karena data yang hilang dapat disebabkan oleh beberapa karakteristik nasabah seperti: jenis kelamin (`gender`), usia (`dob_years`), dan juga jenis pendapatan (`income_type`) yang dapat diasumsikan sebagai jenis atau status pekerjaan.

Contohnya: seorang nasabah berusia muda (20tahun), dengan jenis kelamin wanita, telah menikah dan tidak memiliki pekerjaan, tapi punya jenis pendapatan dari pendapatan (gaji) pasangannya, maka data untuk kolom `days_employed` dan `total_income` bisa saja terdapat nilai yang hilang karena tidak diisi oleh nasabah tersebut.



Langkah-langkah yang akan diambil untuk penyelidikan lanjutan adalah:

1. Melakukan pemeriksaan terhadap kolom-kolom pada dataset untuk melihat apakah ada kejanggalan (nilai yang hilang, duplikasi, dan kesalahan penulisan) yang dapat mempengaruhi analisis data atau tidak.

2. Jika ditemukan kejanggalan, maka akan dilakukan perbaikan terhadap kolom yang bermasalah.

3. Untuk kejanggalan yang merupakan data yang tidak valid, maka akan dihapus. Namun jika kejanggalan yang terjadi adalah karena kesalahan input data, maka nilai tersebut akan diganti atau diisi.

In [15]:
df_nan_values=df[df['days_employed'].isna()]
df_nan_values

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [17]:
df_nan_values.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


Untuk saat ini, kemungkinan penyebab hilangnya nilai dalam data dipengaruhi oleh karakteristik nasabah yaitu usia (kolom `dob_years`). Karena melihat dari data tabel yang telah difilter, sepertinya rata-rata usia nasabah sudah bukan usia produktif, jadi mungkin saja nasabah tersebut tidak memiliki pendapatan (tidak memiliki nilai pada kolom `total_income`) atau sudah tidak bekerja (tidak memiliki nilai pada kolom `days_employed`).

Menurut saya, kemungkinan hal ini yang menyebabkan nilai yang hilang memiliki pola tertentu.

In [19]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Kesimpulan sementara**

Distribusi dalam dataset yang asli mirip dengan distribusi tabel yang telah difilter.
Hal ini dapat menunjukkan bahwa seharusnya filter yang digunakan cukup valid dan tidak mempengaruhi distribusi data secara signifikan, sehingga dapat dikatakan tidak ditemukan outlier yang signifikan pada dataset yang kita miliki.

Namun, sepertinya kita masih perlu melihat faktor-faktor lainnya untuk mendukung pernyataan diatas, seperti melihat pengaruh jenis pendapatan (jenis pekerjaan) atau kolom `income_type` terhadap nilai yang hilang pada kolom `total_income` dan `days_employed`.

In [21]:
df_nan_values['dob_years'].value_counts()

34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
57    56
58    56
54    55
38    54
56    54
37    53
52    53
39    51
33    51
50    51
51    50
45    50
49    50
29    50
43    50
46    48
55    48
48    46
53    44
44    44
60    39
61    38
62    38
64    37
32    37
27    36
23    36
26    35
59    34
63    29
25    23
24    21
66    20
65    20
21    18
22    17
67    16
0     10
68     9
69     5
20     5
71     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64

In [22]:
df_nan_values.groupby('income_type')['dob_years'].count().sort_values()

income_type
entrepreneur        1
civil servant     147
retiree           413
business          508
employee         1105
Name: dob_years, dtype: int64

In [23]:
df.groupby('income_type')['dob_years'].count().sort_values()

income_type
paternity / maternity leave        1
student                            1
entrepreneur                       2
unemployed                         2
civil servant                   1459
retiree                         3856
business                        5085
employee                       11119
Name: dob_years, dtype: int64

**Kesimpulan sementara**

Setelah dilakukan analisis pada kolom `dob_years`(usia), dapat dikatakan bahwa asumsi awal tidak sepenuhnya benar, karena ternyata terdapat beragam usia dengan rentang yang jauh, sehingga nilai yang hilang berdasarkan karakteristik usia merupakan nilai yang bersifat acak tanpa ada pola tertentu.


In [24]:
# Periksa pola lainnya - jelaskan pola tersebut

In [25]:
df_nan_values['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [26]:
df['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [27]:
df.groupby('income_type')['total_income'].count()

income_type
business                        4577
civil servant                   1312
employee                       10014
entrepreneur                       1
paternity / maternity leave        1
retiree                         3443
student                            1
unemployed                         2
Name: total_income, dtype: int64

Saya mencoba melakukan pengelompokkan berdasarkan karakteristik lain, yaitu jenis pendapatan atau jenis pekerjaan (kolom `income_type`), dengan asumsi awal, mungkin nilai yang hilang muncul karena jenis pendapatan tidak tetap (seperti *retiree*, *student*).

Namun ternyata didapatkan bahwa nilai yang hilang justru muncul pada kategori atau jenis pekerjaan yang diperkirakan memiliki pendapatan tetap (contohnya *employee*), meskipun terdapat juga dari sumber pendapatan yang bisa dikategorikan tidak memiliki pendapatan tetap (seperti *retiree* atau *business*).


**Kesimpulan**

Setelah melakukan pemrosesan data tahap awal, tidak ditemukan suatu pola tertentu terhadap nilai yang hilang, sehingga dapat dikatakan bahwa nilai yang hilang tersebut terjadi secara acak.

Hal ini berdasarkan pemeriksaan terhadap kolom yang diasumsikan merupakan faktor yang mempengaruhi nilai yang hilang, yaitu:

1. Setelah dilakukan pemeriksaan terhadap kolom `dob_years` tidak ditemukan pola khusus yang mempengaruhi nilai yang hilang pada kolom `days_employed` dan `total_income`.

2. Setelah dilakukan pemeriksaan pada kolom `income_type` tidak ditemukan pola khusus yang mempengaruhi nilai yang hilang pada kolom `days_employed` dan `total_income`.

Berdasarkan kesimpulan tersebut, saya memutuskan untuk mengatasi nilai-nilai yang hilang dengan mengevaluasi data pada kolom `dob_years` dan `income_type` terhadap kedua kolom yang memiliki nilai yang hilang, supaya bisa didapatkan nilai median maupun nilai rata-rata yang nantinya salah satu dari nilai tersebut akan digunakan untuk mengisi nilai yang hilang pada kolom `days_employed` dan `total_income`.

Untuk selanjutnya, saya akan melakukan pemrosesan data lebih lanjut untuk mengatasi berbagai masalah seperti duplikasi, data yang salah, nilai yang hilang, dsb.

## Transformasi data

In [29]:
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [31]:
df['education']=df['education'].str.lower()

In [33]:
df.groupby('education').size()

education
bachelor's degree       5260
graduate degree            6
primary education        282
secondary education    15233
some college             744
dtype: int64

In [35]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [36]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [37]:
df['children'].value_counts(normalize=True)

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64

Setelah melihat distribusi nilai kolom `children`, didapatkan nilai negatif serta nilai 20 pada kolom tersebut, dan setelah dilihat jumlah persentasenya hanya sekitar 0.57% dari keseluruhan dataset.
Melihat persentasenya yang sangat kecil, maka besar kemungkinan munculnya nilai negatif ini terjadi karena kesalahan penginputan data.
Karena itu, data yang bermasalah tersebut akan dihapus.

In [39]:
df=df.drop(df[(df['children'] < 0) | (df['children'] == 20)].index)

In [41]:
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [43]:
df['days_employed'].describe()

count     19240.000000
mean      63159.820777
std      140928.943329
min      -18388.949901
25%       -2747.235601
50%       -1203.934202
75%        -289.740178
max      401755.400475
Name: days_employed, dtype: float64

In [44]:
days_test=(df['days_employed'] < 0).sum() / len(df)
print(f'{days_test:.2%}')

73.87%


Seperti telah disinggung sebelumnya, kolom `days_employed` memiliki masalah karena terdapat nilai negatif di dalamnya. Berdasarkan deskripsi data, seharusnya kolom tersebut tidak mungkin bernilai negatif, namun dari hasil pemeriksaan ternyata didapatkan persentase sebesar 73.90% untuk data yang nilainya bermasalah (nilai negatif). 

Melihat angka persentase yang cukup besar, saya rasa masalah ini kemungkinan terjadi karena kesalahan teknis dalam perhitungan waktu bekerja, misalkan saja kesalahan *entry* data pada rumus perhitungannya. 

Contohnya: A adalah waktu dimulainya bekerja(*start*), sedangkan B adalah waktu berakhirnya(*end*), untuk mendapatkan jumlah waktu bekerja, maka bisa digunakan **rumus B-A**.

*start* (A) = 1990

*end* (B) = 2000

lama bekerja = (B) - (A) = 2000 - 1990 = 10

Kesalahan yang terjadi mungkin karena rumus tersebut inputnya terbalik, yaitu waktu mulai(*start*) menjadi B dan waktu berakhir(*end*) menjadi A, sehingga akan didapatkan nilai negatif meskipun nilai rentangnya sudah benar.

*start* (B) = 1990

*end* (A) = 2000

lama bekerja = (B) - (A) = 1990 - 2000 = -10

Hal inilah yang mungkin menyebabkan munculnya nilai negatif pada kolom `days_employed`.

Untuk mengatasi masalah ini, nilai-nilai negatif yang terdapat pada kolom `days_employed` akan diubah menjadi nilai absolut.

In [46]:
df['days_employed'] = df['days_employed'].abs()

In [48]:
df['days_employed'].describe()

count     19240.000000
mean      67027.691459
std      139130.846446
min          24.141633
25%         927.984311
50%        2195.251592
75%        5556.372075
max      401755.400475
Name: days_employed, dtype: float64

In [50]:
df['dob_years'].describe()

count    21402.000000
mean        43.300206
std         12.579055
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [51]:
df.loc[df['dob_years']== 0].shape

(100, 12)

Setelah dilakukan pemeriksaan terhadap kolom `dob_years`, didapatkan kejanggalan yaitu terdapat nilai 0 pada kolom tersebut. Hal ini dapat mempengaruhi penilitian terhadap data, karena seharusnya nilai 0 tidak dapat diperhitungkan untuk usia nasabah. Untuk itu, data yang bernilai 0 akan dianggap sebagai data yang tidak valid, sehingga perlu dihapus.

In [53]:
df = df[df.dob_years > 0]

In [55]:
df['dob_years'].describe()

count    21302.000000
mean        43.503474
std         12.252843
min         19.000000
25%         33.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [56]:
df['family_status'].value_counts()

married              12254
civil partnership     4139
unmarried             2783
divorced              1179
widow / widower        947
Name: family_status, dtype: int64

In [60]:
df['gender'].value_counts()

F      14083
M       7218
XNA        1
Name: gender, dtype: int64

Terdapat kejanggalan pada nilai di kolom `gender`, yaitu muncul nilai 'XNA'. Namun karena jumlah data yang janggal tersebut hanya sedikit, maka data ini sepertinya dapat dihapus karena dianggap tidak valid atau dapat diabaikan.

In [62]:
df=df.drop(df[df['gender'] == 'XNA'].index)

In [64]:
df['gender'].value_counts()

F    14083
M     7218
Name: gender, dtype: int64

In [66]:
df['income_type'].value_counts()

employee                       10996
business                        5033
retiree                         3819
civil servant                   1447
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [70]:
df.duplicated().sum()

71

In [72]:
df = df.drop_duplicates()

In [74]:
df.duplicated().sum()

0

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21230 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  int64  
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  int64  
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  int64  
 10  total_income      19149 non-null  float64
 11  purpose           21230 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [77]:
df['days_employed'].isna().sum()

2081

In [78]:
df['total_income'].isna().sum()

2081

In [79]:
new_percentage = df['days_employed'].isna().sum() / len(df)
print(f'{new_percentage:.2%}')

9.80%


Setelah melakukan penelitian terhadap beberapa kolom pada dataset, didapatkan sejumlah masalah, seperti:

- Terdapat ejaan alternatif (duplikat implisit) pada kolom `education` dimana terdapat perbedaan penggunaan huruf kapital dan hurup kecil. Untuk itu, dilakukan perubahan dengan cara membuat semua ejaan pada kolom `education` menggunakan huruf kecil saja.


- Pada kolom `children` terdapat nilai minus(negatif) dan nilai 20 (nilai yang abnormal untuk jumlah anak) dengan persentase sebesar 0.57%. Karena persentasenya sangat kecil, data ini dihapus dan dapat diabaikan karena dirasa data tersebut tidak valid.


- Terdapat 101 data dengan usia bernilai 0. Hal ini dirasa tidak valid, karena usia 0 tidak dapat diperhitungkan untuk analisis data nasabah, karena itu data ini sudah dihapus.


- Pada kolom `days_employed` terdapat nilai negatif dengan persentase sebesar 73.90%. Karena persentase yang cukup besar dan dapat mempengaruhi analisis data, serta karena diasumsikan masalah ini terjadi karena kesalahan teknis saat *entry* data, maka data tersebut dirubah menjadi nilai absolut.


- Terdapat 1 nilai unik di kolom `gender` yaitu 'XNA', dan data tersebut sudah dihapus karena dianggap tidak valid.


- Terdapat 71 duplikat dalam dataset dan duplikasi tersebut sudah dihapus.


Dengan perubahan yang dilakukan seperti penjelasan diatas, maka didapatkan dataset baru dengan total data 21230. Namun masih terdapat masalah dengan nilai yang hilang di kolom `days_employed` dan `total_income` sebesar 2081 atau 9.80% dari keseluruhan dataset.

# Bekerja dengan nilai yang hilang

Terdapat 2 kolom pada dataset yang memiliki ID, yaitu: `education` dan `family_status`. ID kedua kolom tersebut juga sudah dibuat menjadi kolom masing-masing tersendiri, yaitu: `education_id` dan `family_status_id`.

Saya akan menggunakan dictionary dengan key berupa **string** (status pendidikan dan keluarga seperti yang ada dalam kolom `education` dan `family_status`) yang digunakan untuk mengidentifikasikan **value** kolom `education_id` dan `family_status_id`. Jenis dictionary ini saya pilih untuk mempermudah memetakan pembagian status pendidikan (*education*) dan keluarga (*family_status*).

In [81]:
education_dictionary = df[['education', 'education_id']]
education_dictionary = education_dictionary.drop_duplicates()
education_dictionary

,education,education_id
0,bachelor's degree,0
1,secondary education,1
13,some college,2
31,primary education,3
2963,graduate degree,4


In [82]:
dict_test = dict(zip(df['education'], df['education_id']))
dict_test

{"bachelor's degree": 0,
 'secondary education': 1,
 'some college': 2,
 'primary education': 3,
 'graduate degree': 4}

In [83]:
family_dictionary = df[['family_status', 'family_status_id']]
family_dictionary = family_dictionary.drop_duplicates()
family_dictionary

,family_status,family_status_id
0,married,0
4,civil partnership,1
18,widow / widower,2
19,divorced,3
24,unmarried,4


In [84]:
dict_fam_test = dict(zip(df['family_status'], df['family_status_id']))
dict_fam_test

{'married': 0,
 'civil partnership': 1,
 'widow / widower': 2,
 'divorced': 3,
 'unmarried': 4}

### Memperbaiki nilai yang hilang di `total_income`

Seperti yang dapat kita lihat setelah melakukan perbaikan pada beberapa kolom pada dataset, masih didapatkan nilai yang hilang untuk kolom `days_employed` dan `total_income` sebanyak 2081 atau 9.80% dari keseluruhan data. 

Untuk itu, langkah yang perlu dilakukan adalah memperbaiki nilai yang hilang pada kedua kolom tersebut.

Langkah pertama, saya akan membuat kategori usia nasabah seperti berikut:

- Nasabah dengan usia kurang dari 18 tahun tidak diperhitungkan.
- Nasabah dengan rentang usia antara 18 tahun sampai 24 tahun dikategorikan sebagai usia muda atau `young`
- Nasabah dengan rentang usia antara 25 hingga 44 tahun dikategorikan sebagai dewasa atau `adult`
- Nasabah dengan rentang usia 45 sampai dengan 59 tahun dikategorikan sebagai usia matang atau `mature`
- Nasabah dengan usia 60 tahun keatas dikategorikan sebagai orang tua atau `old`

Untuk langkah selanjutnya, saya akan menggunakan kategori usia tersebut untuk mencari total pendapatan per kategori usia.

In [86]:
def dob_years_group(dob_years):
    if 18 <= dob_years <= 24:
        return 'young'
    if 25 <= dob_years <= 44:
        return 'adult'
    if 45 <= dob_years <= 59:
        return 'mature'
    return 'old'

In [88]:
dob_years_group(25)

'adult'

In [90]:
df['dob_years_group']=df['dob_years'].apply(dob_years_group)

In [93]:
new_df = df.dropna()
new_df.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,mature
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,mature
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult


In [95]:
new_df.groupby('dob_years_group')['total_income'].mean().sort_values()

dob_years_group
young     22717.058884
old       23015.440182
mature    26667.490725
adult     28081.895167
Name: total_income, dtype: float64

In [97]:
new_df.groupby('dob_years_group')['total_income'].median().sort_values()

dob_years_group
old       19732.9470
young     20574.8955
mature    23070.5710
adult     24412.5800
Name: total_income, dtype: float64

In [98]:
print(new_df.groupby('income_type')['total_income'].mean().sort_values())
print()
new_df.groupby('income_type')['total_income'].median().sort_values()

income_type
paternity / maternity leave     8612.661000
student                        15712.260000
unemployed                     21014.360500
retiree                        21950.722935
employee                       25822.872585
civil servant                  27336.442546
business                       32424.420789
entrepreneur                   79866.103000
Name: total_income, dtype: float64



income_type
paternity / maternity leave     8612.6610
student                        15712.2600
retiree                        18959.6260
unemployed                     21014.3605
employee                       22815.1035
civil servant                  24076.1150
business                       27594.6410
entrepreneur                   79866.1030
Name: total_income, dtype: float64

In [99]:
new_df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19149.000000,19149.000000,19149.000000,19149.000000,19149.000000,19149.000000,19149.000000
mean,0.473080,67015.855715,43.468432,0.819573,0.972113,0.081153,26798.699010
std,0.750161,139122.608192,12.257193,0.550728,1.420571,0.273077,16515.512928
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,927.789271,33.000000,1.000000,0.000000,0.000000,16484.526000
50%,0.000000,2197.460718,43.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,5558.591054,53.000000,1.000000,1.000000,0.000000,32536.555000
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Jika diperhatikan, hampir semua kolom pada dataset merupakan faktor yang dapat mempengaruhi pendapatan, seperti : usia, lama waktu bekerja atau pengalaman bekerja, pendidikan, sumber pendapatan (jenis pekerjaan), dan *family status*. 

Namun merujuk pada data diatas, kemungkinan besar karakteristik yang paling menentukan pendapatan adalah kelompok usia, lama atau waktu bekerja, dan sumber atau jenis pendapatan. Umumnya, kelompok usia produktif dengan pengalaman bekerja yang lebih lama dapat menghasilkan nilai pendapatan yang lebih tinggi. Sedangkan untuk jenis pendapatan, jenis pendapatan yang berasal dari jenis pekerjaan tetap (contohnya seperti *employee*) pasti akan memberikan nilai yang jelas pada total pendapatan.

Karena kolom `days_employed` sendiri masih memiliki nilai yang hilang dan dirasa nilainya terlalu beragam, maka saya putuskan untuk menggunakan kolom `dob_years_group` dan `income_type` sebagai karakteristik yang paling menentukan pendapatan.

Untuk mengisi nilai yang hilang pada kolom `total_income`, saya akan menggunakan median, karena biasanya rata-rata (mean) dapat dipengaruhi oleh nilai-nilai ekstrim atau outlier, sehingga jika data memiliki nilai-nilai ekstrim yang tinggi atau rendah, median mungkin lebih sesuai untuk digunakan.

In [101]:
median_income_pivot = new_df.pivot_table(index='income_type', columns='dob_years_group', values='total_income', aggfunc='median', margins=True)
median_income_pivot

dob_years_group,adult,mature,old,young,All
income_type,,,,,
business,27789.7755,28323.9125,29171.9890,22189.0780,27594.6410
civil servant,24328.8550,23847.2850,23957.6405,21361.7300,24076.1150
employee,23050.7085,22779.2350,23365.6140,19725.3590,22815.1035
entrepreneur,79866.1030,NaN,NaN,NaN,79866.1030
paternity / maternity leave,8612.6610,NaN,NaN,NaN,8612.6610
retiree,19546.0750,19709.0600,18412.9250,14298.9760,18959.6260
student,NaN,NaN,NaN,15712.2600,15712.2600
unemployed,9593.1190,32435.6020,NaN,NaN,21014.3605
All,24412.5800,23070.5710,19732.9470,20574.8955,23202.8700


In [102]:
def get_median_total_income(row):  
    dob_years_group= row['dob_years_group']
    income_type = row['income_type']
    try:
        return median_income_pivot[dob_years_group][income_type]
    except:
        return 'error'

In [104]:
df['median_total_income'] = df.apply(get_median_total_income, axis=1)

In [105]:
df.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult,23050.7085
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,23050.7085
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,23050.7085
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,23050.7085
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,mature,19709.0600
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult,27789.7755
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult,27789.7755
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,mature,22779.2350
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult,23050.7085
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult,23050.7085


In [107]:
df[df['median_total_income'] == 'error']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income


In [109]:
df['total_income'] = df['total_income'].fillna(df['median_total_income'])

In [110]:
df.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult,23050.7085
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,23050.7085
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,23050.7085
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,23050.7085
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,mature,19709.0600
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult,27789.7755
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult,27789.7755
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,mature,22779.2350
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult,23050.7085
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult,23050.7085


In [112]:
df['total_income'].shape

(21230,)

In [113]:
df['total_income'].isna().sum()

1

In [114]:
df.loc[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,mature,NaN


Setelah mencoba mengisi nilai yang hilang pada kolom `total_income` dengan nilai median, ternyata masih didapatkan 1 data yang masih bernilai NaN. Hal ini disebabkan karena tidak terdapat nilai median untuk kategori kelompok usia dan sumber pendapatannya (seperti terlihat pada tabel `median_income_pivot`).

Karena data yang masih bernilai NaN ini hanya berjumlah 1 data, maka saya putuskan untuk mengisi nilai NaN tersebut menjadi 0.

In [115]:
df['total_income'] = df['total_income'].fillna(0)

In [116]:
df['median_total_income'] = df['median_total_income'].fillna(0)

In [117]:
df['total_income'].isna().sum()

0

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21230 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21230 non-null  int64  
 1   days_employed        19149 non-null  float64
 2   dob_years            21230 non-null  int64  
 3   education            21230 non-null  object 
 4   education_id         21230 non-null  int64  
 5   family_status        21230 non-null  object 
 6   family_status_id     21230 non-null  int64  
 7   gender               21230 non-null  object 
 8   income_type          21230 non-null  object 
 9   debt                 21230 non-null  int64  
 10  total_income         21230 non-null  float64
 11  purpose              21230 non-null  object 
 12  dob_years_group      21230 non-null  object 
 13  median_total_income  21230 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.4+ MB


###  Memperbaiki nilai di `days_employed`

In [120]:
new_df.groupby('income_type')['days_employed'].median().describe()

count         8.000000
mean      92735.225177
std      168567.672692
min         520.848083
25%        1311.683133
50%        2123.347501
75%       93789.845075
max      366413.652744
Name: days_employed, dtype: float64

In [121]:
new_df.groupby('dob_years_group')['days_employed'].median().describe()

count         4.000000
mean      90218.895224
std      176677.241874
min         742.595507
25%        1336.003504
50%        2451.683584
75%       91334.575304
max      355229.618218
Name: days_employed, dtype: float64

In [123]:
new_df.groupby('income_type')['days_employed'].mean().describe()

count         8.000000
mean      92960.608936
std      168357.958538
min         520.848083
25%        1734.322538
50%        2811.250427
75%       93803.460978
max      366413.652744
Name: days_employed, dtype: float64

In [124]:
new_df.groupby('dob_years_group')['days_employed'].mean().describe()

count         4.000000
mean      96287.473029
std      133609.003958
min        1282.579590
25%        3954.814487
50%       48720.325762
75%      141052.984305
max      286426.661002
Name: days_employed, dtype: float64

Untuk mengisi nilai yang hilang pada kolom `days_employed`, saya akan menggunakan median, karena melihat dari distribusi data diatas, sepertinya terdapat nilai-nilai ekstrim atau outlier, sehingga jika data memiliki nilai-nilai ekstrim yang tinggi atau rendah, median mungkin lebih sesuai untuk digunakan daripada rata-rata.

In [126]:
median_days_employed_pivot = new_df.pivot_table(index='income_type', columns='dob_years_group', values='days_employed', aggfunc='median', margins=True)
median_days_employed_pivot

dob_years_group,adult,mature,old,young,All
income_type,,,,,
business,1436.968328,2060.920732,2470.912766,691.676752,1555.993659
civil servant,2460.012794,3672.026905,3318.440092,922.372533,2672.903939
employee,1476.620047,2119.093670,2669.073965,764.870742,1573.791064
entrepreneur,520.848083,NaN,NaN,NaN,520.848083
paternity / maternity leave,3296.759962,NaN,NaN,NaN,3296.759962
retiree,362741.382924,364914.419761,365658.028700,334764.259831,365269.100414
student,NaN,NaN,NaN,578.751554,578.751554
unemployed,337524.466835,395302.838654,NaN,NaN,366413.652744
All,1533.806170,3369.560999,355229.618218,742.595507,2197.460718


In [127]:
def get_median_days_employed(row):  
    dob_years_group= row['dob_years_group']
    income_type = row['income_type']
    try:
        return median_days_employed_pivot[dob_years_group][income_type]
    except:
        return 'error'

In [130]:
df['median_days_employed']= df.apply(get_median_days_employed, axis=1)

In [132]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income,median_days_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult,23050.7085,1476.620047
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,23050.7085,1476.620047
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,23050.7085,1476.620047
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,23050.7085,1476.620047
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,mature,19709.0600,364914.419761
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult,27789.7755,1436.968328
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult,27789.7755,1436.968328
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,mature,22779.2350,2119.093670
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult,23050.7085,1476.620047
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult,23050.7085,1476.620047


In [134]:
df['days_employed'] = df['days_employed'].fillna(df['median_days_employed'])

In [135]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income,median_days_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult,23050.7085,1476.620047
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,23050.7085,1476.620047
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,23050.7085,1476.620047
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,23050.7085,1476.620047
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,mature,19709.0600,364914.419761
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult,27789.7755,1436.968328
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult,27789.7755,1436.968328
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,mature,22779.2350,2119.093670
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult,23050.7085,1476.620047
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult,23050.7085,1476.620047


In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21230 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21230 non-null  int64  
 1   days_employed         21229 non-null  float64
 2   dob_years             21230 non-null  int64  
 3   education             21230 non-null  object 
 4   education_id          21230 non-null  int64  
 5   family_status         21230 non-null  object 
 6   family_status_id      21230 non-null  int64  
 7   gender                21230 non-null  object 
 8   income_type           21230 non-null  object 
 9   debt                  21230 non-null  int64  
 10  total_income          21230 non-null  float64
 11  purpose               21230 non-null  object 
 12  dob_years_group       21230 non-null  object 
 13  median_total_income   21230 non-null  float64
 14  median_days_employed  21229 non-null  float64
dtypes: float64(4), int6

In [138]:
df.loc[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income,median_days_employed
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,0.0,buy residential real estate,mature,0.0,NaN


Setelah mencoba mengisi nilai yang hilang pada kolom `days_employed` dengan nilai median, ternyata masih didapatkan 1 data yang masih bernilai NaN. Hal ini disebabkan karena tidak terdapat nilai median untuk kategori kelompok usia dan sumber pendapatannya (seperti terlihat pada tabel `median_days_employed_pivot`).

Karena data yang masih bernilai NaN ini hanya berjumlah 1 data, maka saya putuskan untuk mengisi nilai NaN tersebut menjadi 0.

In [139]:
df['days_employed'] = df['days_employed'].fillna(0)

In [140]:
df['median_days_employed'] = df['median_days_employed'].fillna(0)

In [141]:
df['days_employed'].isna().sum()

0

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21230 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21230 non-null  int64  
 1   days_employed         21230 non-null  float64
 2   dob_years             21230 non-null  int64  
 3   education             21230 non-null  object 
 4   education_id          21230 non-null  int64  
 5   family_status         21230 non-null  object 
 6   family_status_id      21230 non-null  int64  
 7   gender                21230 non-null  object 
 8   income_type           21230 non-null  object 
 9   debt                  21230 non-null  int64  
 10  total_income          21230 non-null  float64
 11  purpose               21230 non-null  object 
 12  dob_years_group       21230 non-null  object 
 13  median_total_income   21230 non-null  float64
 14  median_days_employed  21230 non-null  float64
dtypes: float64(4), int6

## Pengkategorian Data

In [144]:
df['purpose'].value_counts()

wedding ceremony                            785
having a wedding                            759
to have a wedding                           755
real estate transactions                    669
buy commercial real estate                  655
buying property for renting out             647
transactions with commercial real estate    643
housing transactions                        641
purchase of the house for my family         636
housing                                     635
purchase of the house                       634
property                                    627
construction of own property                626
transactions with my real estate            623
building a property                         619
purchase of my own house                    618
building a real estate                      617
buy real estate                             612
housing renovation                          602
buy residential real estate                 599
buying my own car                       

In [146]:
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Kolom `purpose` memiliki nilai unik dengan duplikat implisit yang cukup banyak. Hal ini mungkin karena nasabah menuliskan tujuannya berdasarkan asumsi pribadi masing-masing, sehingga meskipun tujuannya sama, tapi dapat menggunakan kalimat atau kata yang berbeda.

Dilihat berdasarkan nilai uniknya, tujuan pinjaman untuk kebutuhan *property* (baik untuk pembelian rumah, gedung atau *real estate*) memiliki jumlah yang paling besar diantara tujuan lainnya.

Saya akan membuat 4 kategori umum berdasarkan kolom `purpose` untuk memudahkan pembagian kelompok tujuan, yaitu:

1. `house` : untuk semua value dalam kolom `purpose` yang berhubungan dengan rumah (pembelian, sewa, perbaikan), *real estate*, dan *property*.
2. `car` : untuk semua value dalam kolom `purpose` yang berhubungan dengan kendaraan.
3. `wedding` : untuk semua value dalam kolom `purpose` yang berhubungan dengan kebutuhan untuk pernikahan.
4. `education` : untuk semua value dalam kolom `purpose` yang berhubungan dengan pendidikan (semua jenis dan jenjang pendidikan)

In [148]:
purpose_category = df['purpose'].apply(lambda x: 'house' if 'house' in x else
                                       ('car' if 'car' in x else
                                        ('wedding' if 'wedding' in x else
                                         ('education' if 'education' in x else
                                          ('house' if 'housing' in x else
                                           ('house' if 'property' in x else
                                            ('house' if 'real estate' in x else
                                             ('education' if 'university' in x else
                                              ('education' if 'educated' in x else'other')))))))))

In [150]:
df['purpose_category'] = purpose_category

In [151]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income,median_days_employed,purpose_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult,23050.7085,1476.620047,house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,23050.7085,1476.620047,car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,23050.7085,1476.620047,house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,23050.7085,1476.620047,education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,mature,19709.0600,364914.419761,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,adult,27789.7755,1436.968328,house
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,old,18412.9250,365658.028700,car
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult,23050.7085,1476.620047,house
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adult,23050.7085,1476.620047,car


In [152]:
df['purpose_category'].value_counts()

house        10703
car           4258
education     3970
wedding       2299
Name: purpose_category, dtype: int64

In [154]:
df['total_income'].describe()

count     21230.000000
mean      26460.298219
std       15747.169047
min           0.000000
25%       17203.075250
50%       23050.708500
75%       31327.351000
max      362496.645000
Name: total_income, dtype: float64

In [155]:
df['days_employed'].describe()

count     21230.000000
mean      67123.577551
std      139247.355005
min           0.000000
25%        1010.484209
50%        2119.093670
75%        5338.271214
max      401755.400475
Name: days_employed, dtype: float64

Untuk data numerik, terdapat 3 kolom yaitu `dob_years`, `total_income` dan `days_employed`. Sebelumnya kolom `dob_years` sudah dikelompokkan, sedangkan kolom `days_employed` sulit untuk dikategorikan karena tidak diketahui satuan waktunya secara jelas (apakah angkanya dalam hitungan menit, jam, atau hari), maka saya akan membuat kategori untuk kolom `total_income` saja.

Kolom `total_income` akan saya bagi ke dalam beberapa kategori, yaitu:

1. total pendapatan antara 0 sampai dengan 55000 termasuk kategori low
2. total pendapatan antara 55001 sampai dengan 150000 termasuk kategori middle
3. total pendapatan antara 150001 sampai dengan 350000 termasuk kategori upper-middle
4. total pendapatan 350001 keatas termasuk kategori high/rich

In [158]:
def income_group(total_income):
    if 0 <= total_income <= 55000:
        return 'low'
    if 55001 <= total_income <= 150000:
        return 'middle'
    if 150001 <= total_income <= 350000:
        return 'upper-middle'
    return 'high/rich'

In [160]:
df['income_group']=df['total_income'].apply(income_group)


In [162]:
df['income_group'].value_counts()

low             20298
middle            904
upper-middle       26
high/rich           2
Name: income_group, dtype: int64

## Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan probabilitas melakukan gagal bayar pinjaman?**

In [164]:
pivot_table_children = df.pivot_table(index='children', columns='debt', values='dob_years', aggfunc='count')
pivot_table_children['insolvency_percentage'] = pivot_table_children[1] / (pivot_table_children[1] + pivot_table_children[0]) * 100
pivot_table_children


debt,0,1,insolvency_percentage
children,,,
0,12963.0,1058.0,7.545824
1,4351.0,441.0,9.202838
2,1845.0,194.0,9.514468
3,301.0,27.0,8.231707
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


In [165]:
df[df['children'] == 5]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income,median_days_employed,purpose_category,income_group
3979,5,1476.620047,42,secondary education,1,civil partnership,1,M,employee,0,23050.7085,buying my own car,adult,23050.7085,1476.620047,car,low
4397,5,3248.839837,36,secondary education,1,married,0,F,business,0,26953.7480,real estate transactions,adult,27789.7755,1436.968328,house,low
7866,5,773.124856,36,secondary education,1,married,0,F,employee,0,7803.6630,housing transactions,adult,23050.7085,1476.620047,house,low
15822,5,418.199982,31,secondary education,1,married,0,F,employee,0,12408.3890,second-hand car purchase,adult,23050.7085,1476.620047,car,low
15916,5,2286.262752,37,secondary education,1,married,0,F,employee,0,41071.7360,buy real estate,adult,23050.7085,1476.620047,house,low
16211,5,387.317579,35,secondary education,1,civil partnership,1,F,civil servant,0,20176.3440,having a wedding,adult,24328.8550,2460.012794,wedding,low
20452,5,268.425464,38,primary education,3,married,0,F,employee,0,34007.2590,going to university,adult,23050.7085,1476.620047,education,low
20837,5,2386.600221,35,secondary education,1,married,0,F,business,0,32678.7030,housing,adult,27789.7755,1436.968328,house,low
21156,5,1690.018117,59,secondary education,1,married,0,M,employee,0,43050.9360,transactions with my real estate,mature,22779.2350,2119.093670,house,low


**Kesimpulan**

Setelah memeriksa data anak dengan data gagal bayar pinjaman dan menghitung persentasenya, didapatkan kesimpulan sebagai berikut:

 1. Persentase gagal bayar terbanyak berada di kategori nasabah yang memiliki 4 anak.
 2. Persentase gagal bayar kedua terbanyak berada di kategori nasabah yang memiliki 2 anak.
 3. Nasabah yang memiliki 5 anak tidak pernah gagal bayar. 
    
Dengan demikian jumlah anak tidak selalu mempengaruhi persentase gagal bayar seorang nasabah, sehingga hipotesis awal terbukti tidak sepenuhnya benar.

Namun, jika kita melihat tabel yang telah difilter berdasarkan nasabah yang memiliki 5 anak, dapat dilihat kategori pendapatan mereka berada di kelas `very low/poor` dan `low` sehingga, mungkin saja masih dapat terjadi gagal bayar. Jadi nasabah yang termasuk kategori memiliki 5 anak masih perlu dievaluasi lebih lanjut.

**Apakah terdapat korelasi antara status keluarga dengan probabilitas melakukan gagal bayar pinjaman?**

In [167]:
pivot_table_family = df.pivot_table(index='family_status', columns='debt', values='dob_years', aggfunc='count')
pivot_table_family['insolvency_percentage'] = pivot_table_family[1] / (pivot_table_family[1] + pivot_table_family[0]) * 100
pivot_table_family

debt,0,1,insolvency_percentage
family_status,,,
civil partnership,3729,383,9.314202
divorced,1095,84,7.124682
married,11290,923,7.557521
unmarried,2508,272,9.784173
widow / widower,884,62,6.553911


**Kesimpulan**

Setelah memeriksa data status perkawinan dengan data gagal bayar pinjaman dan menghitung persentasenya, dapat diambil kesimpulan bahwa:
1. Nasabah dengan status *unmarried* atau tidak/belum menikah atau bisa dikatakan hidup sendiri memiliki persentase gagal bayar paling tinggi sebanyak 9.78%.
2. Nasabah dengan status *civil partnership* memiliki persentase gagal bayar kedua tertinggi sebanyak 9.3%.

Dengan demikian, dapat dikatakan bahwa hipotesa di awal terbukti benar. Status perkawinan dapat mempengaruhi stabilitas keuangan nasabah, sehingga nasabah dengan status menikah dapat diasumsikan memiliki keuangan yang lebih stabil dibandingkan nasabah yang tidak/belum menikah.

**Apakah terdapat korelasi antara tingkat pendapatan dengan probabilitas melakukan gagal bayar pinjaman?**

In [169]:
pivot_table_income = df.pivot_table(index='income_group', columns='debt', values='dob_years', aggfunc='count')
pivot_table_income['insolvency_percentage'] = pivot_table_income[1] / (pivot_table_income[1] + pivot_table_income[0]) * 100
pivot_table_income

debt,0,1,insolvency_percentage
income_group,,,
high/rich,1,1,50.000000
low,18636,1662,8.187999
middle,844,60,6.637168
upper-middle,25,1,3.846154


In [170]:
df[df['income_group'] == 'high/rich']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,median_total_income,median_days_employed,purpose_category,income_group
12412,0,1477.438114,44,bachelor's degree,0,married,0,M,business,0,362496.645,housing renovation,adult,27789.7755,1436.968328,house,high/rich
19606,1,2577.664662,39,bachelor's degree,0,married,0,M,business,1,352136.354,building a property,adult,27789.7755,1436.968328,house,high/rich


**Kesimpulan**

Hipotesis ini sebagian terbukti tidak benar.

1. Nasabah dengan total pendapatan tinggi (kategori `high/rich`) memiliki persentase gagal bayar sebanyak 50%. Karena dari 2 nasabah di kategori ini, salah satunya pernah mengalami gagal bayar.
2. Namun nasabah yang mempunyai total pendapatan rendah (kategori `low`) ternyata memiliki persentase gagal bayar tertinggi kedua yaitu sebanyak 8.18%. Sehingga masih dapat diasumsikan bahwa total pendapatan yang rendah dapat berpengaruh terhadap probabilitas melakukan gagal bayar.

Data nasabah yang memiliki pendapatan tinggi sebenarnya kurang bisa dijadikan acuan untuk pengujian hipotesis ini, karena hanya berjumlah 2 orang dibandingkan kategori lainnya. Jadi bisa dikatakan sebagian hipotesis tidak benar.

**Bagaimana tujuan kredit memengaruhi persentase gagal bayar?**

In [172]:
pivot_table_purpose = df.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc='count')
pivot_table_purpose['insolvency_percentage'] = pivot_table_purpose[1] / (pivot_table_purpose[1] + pivot_table_purpose[0]) * 100
pivot_table_purpose

debt,0,1,insolvency_percentage
purpose_category,,,
car,3861,397,9.323626
education,3601,369,9.294710
house,9926,777,7.259647
wedding,2118,181,7.872988


**Kesimpulan**

Data yang didapatkan menunjukkan bahwa:

1. Nasabah dengan tujuan kredit untuk kebutuhan jangka pendek seperti kebutuhan pembelian mobil mempunyai persentase gagal bayar terbesar, yaitu 9.3%. 
2. Sedangkan untuk tujuan melangsungkan pernikahan, yang diasumsikan termasuk kebutuhan jangka pendek, ternyata memiliki persentase tidak terlalu tinggi, yaitu 7.2%.
3. Nasabah dengan tujuan kredit untuk investasi jangka panjang seperti pembelian rumah atau *property* ternyata lebih jarang terjadi gagal bayar. 
4. Namun untuk tujuan pendidikan, yang dapat diasumsikan sebagai kebutuhan jangka panjang, ternyata memiliki persentase yang cukup tinggi sebesar 9.2%.

Tujuan kredit untuk kebutuhan/investasi berdasarkan jangka waktu tertentu (jangka panjang atau pendek) tidak terlalu berpengaruh terhadap probabilitas gagal bayar, karena terbukti memiliki persentase yang kurang lebih sama nilainya, sehingga baik tujuan dalam jangka panjang dan pendek masing-masing probabilitasnya sama besarnya untuk terjadi gagal bayar.

# Kesimpulan umum 

Kita telah mencoba mengolah data untuk menguji apakah hipotesis kita terbukti benar atau tidak.

Dari tahap eksplorasi data, transformasi data dan pengkategorian data dapat disimpulkan:

1. Terdapat nilai-nilai yang hilang pada 2 kolom dataset, yaitu kolom `days_employed` dan `total_income`.
2. Terdapat data yang bermasalah seperti nilai abnormal (contohnya: nilai negatif atau nilai yang terlampau besar maupun kecil sehingga tidak normal) pada kolom `days_employed`, `dob_years`, `gender`, dan `children`. 
3. Data bermasalah yang terjadi sebagian besar diasumsikan terjadi karena kesalahan *input* data. Masing-masing nilai yang bermasalah tersebut sudah ditangani dengan cara:
    - Untuk data bermasalah yang memiliki persentase kecil (di bawah 1%), data dianggap tidak valid dan sudah dihapus.
    - Untuk data bermasalah yang memiliki persentase besar (di atas 70%), nilainya sudah diperbaiki.


4. Terdapat data yang terduplikasi, namun duplikat tersebut telah dihapus sehingga didapatkan total data yang baru.
5. Nilai yang hilang pada kedua kolom dataset, yaitu `days_employed` dan `total_income` dipengaruhi oleh karakteristik nasabah lainnya, seperti usia (`dob_years`) dan juga jenis pendapatan (`income_type`). Nilai yang hilang tersebut telah diisi dengan nilai median dari distribusi data masing-masing kolom `days_employed` dan `total_income` dengan karakteristik usia dan jenis pendapatan yang telah dikelompokkan.
6. Semua data bermasalah telah ditangani dan siap digunakan untuk pengujian hipotesis.

Pada awal proyek, kita telah membuat hipotesis sebagai berikut:


1. Jumlah anak yang dimiliki oleh nasabah dapat mempengaruhi probabilitas gagal bayar dalam pelunasan pinjaman. Semakin banyak anak yang dimiliki, probabilitasnya semakin tinggi.

2. Status perkawinan biasanya berpengaruh terhadap stabilitas keuangan nasabah sehingga nasabah dengan status sudah menikah dapat memiliki probabilitas gagal bayar lebih rendah dibanding yang belum menikah atau hidup sendiri.

3. Nasabah dengan tingkat pendapatan rendah dapat mempunyai probabilitas gagal bayar yang lebih besar dibandingkan dengan nasabah dengan tingkat pendapatan tinggi.

4. Tujuan pinjaman untuk kebutuhan/investasi jangka panjang (contoh: membeli rumah atau *property*) dapat mempunyai probabilitas gagal bayar yang lebih besar dibandingkan dengan nasabah yang memiliki tujuan pinjaman untuk kebutuhan jangka pendek (contoh: untuk biaya menikah).

Setelah menganalisis data yang tersedia, kita dapat menyimpulkan bahwa:

1. Nasabah dengan jumlah anak paling banyak tidak pernah mengalami gagal bayar. 

Hipotesis ini tidak dapat kita terima. Namun, data nasabah tersebut perlu dievaluasi lebih lanjut.

2. Nasabah dengan status *unmarried* dan *civil partnership* memiliki persentase gagal bayar cukup besar. Mereka diasumsikan tidak menikah atau hidup sendiri atau tidak terikat status perkawinan yang sah.

Hipotesis kedua dapat diterima sepenuhnya.

3. Nasabah dengan total pendapatan tinggi (kategori `high/rich`) memiliki persentase gagal bayar yang tinggi.

Hipotesis ini sebagian tidak dapat diterima. Nasabah dengan total pendapatan rendah juga terbukti memiliki persentase gagal bayar yang tinggi. Data nasabah berpenghasilan tinggi juga jumlahnya terlalu sedikit, sehingga kurang layak dijadikan acuan.

4. Tujuan kredit untuk kebutuhan/investasi berdasarkan jangka waktu tertentu (jangka panjang atau pendek) memiliki probabilitas terjadi gagal bayar yang sama besar.

Hipotesis ini ditolak. Dengan memiliki probabilitas sama besar, dapat dikatakan tidak ada pengaruh signifikan antara tujuan kredit dengan terjadinya gagal bayar.

Dengan demikian, kita sudah menguji hipotesis kita dan menarik kesimpulan. 
Untuk penilaian kredit seorang nasabah, kita dapat mengevaluasi data nasabah tersebut dan memberikan pernilaian dengan memperhatikan faktor-faktor berikut:
1. Jumlah anak.
2. Status Perkawinan.
3. Total pendapatan.
